In [1]:
import pandas as pd
import psycopg2

In [2]:
server = 'localhost'
database = 'python_etl_varias_bases'
user = 'postgres'
password = 'root'

connection_string = f"dbname={database} user={user} password={password} host={server}"

try:
    conn = psycopg2.connect(connection_string)
    print("Conexão bem-sucedida ao PostgreSQL!")
    
    cursor = conn.cursor()
except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)

Conexão bem-sucedida ao PostgreSQL!


In [3]:
clientes = pd.read_csv("/home/juniorsilvadev/engenharia-dados/python-etl/Arquivos Excel/arquivos_csv/Clientes.csv", delimiter=',')
produtos = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Arquivos Excel/arquivos_excel/Produto.xlsx")
categorias = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Arquivos Excel/arquivos_excel/Categoria.xlsx")
ordens = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Arquivos Excel/arquivos_excel/Ordens.xlsx")
items = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Arquivos Excel/arquivos_excel/items.xlsx")

In [ ]:
produtos = produtos.rename(columns={
    'ID': 'id',
    'Price': 'price',
    'Id_Category': 'id_category',
    'Name': 'name'
})

In [ ]:
dados = {
    'ordens': ordens,
    'clientes': clientes,
}

transform_for_data_hour = ['ordens', 'clientes']

for table_name in transform_for_data_hour:
    table = dados[table_name]
    table['created_at'] = pd.to_datetime(table['created_at'])
    print(f"Coluna 'created_at' da tabela '{table_name}' convertida com sucesso!")

In [ ]:
clientes['email'] = clientes['email'].fillna('Sem registro')
clientes['street'] = clientes['street'].fillna('Sem infor')
clientes['number'] = clientes['number'].fillna('Sem número')
clientes['additionals'] = clientes['additionals'].fillna('Sem infor')

produtos = produtos.drop_duplicates(subset=['id'])

In [ ]:
create_tables_queries = """
    CREATE TABLE IF NOT EXISTS produtos (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        price NUMERIC(10, 2),
        id_category INTEGER
    );

    CREATE TABLE IF NOT EXISTS categorias (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255)
    );
    
    CREATE TABLE IF NOT EXISTS items (
        id SERIAL PRIMARY KEY,
        order_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        total_price NUMERIC(10, 2)
    );
    
    CREATE TABLE IF NOT EXISTS ordens (
        id SERIAL PRIMARY KEY,
        created_at TIMESTAMP,
        customer_id INTEGER,
        status VARCHAR(255)
    );
    
    CREATE TABLE IF NOT EXISTS clientes (
        id SERIAL PRIMARY KEY,
        created_at TIMESTAMP,
        first_name VARCHAR(255),
        last_name VARCHAR(255),
        email VARCHAR(255),
        cell_phone VARCHAR(255),
        country VARCHAR(255),
        state VARCHAR(255),
        street VARCHAR(255),
        number VARCHAR(255),
        additionals VARCHAR(255)
    );
"""

try:
    cursor.execute(create_tables_queries)
    conn.commit()
    print("Tabelas criadas com sucesso!")
except Exception as e:
    print("Erro ao criar a tabelas:", e)
    conn.rollback()

In [ ]:
tables_to_truncate = ['produtos', 'categorias', 'items', 'ordens', 'clientes']

for table in tables_to_truncate:
    truncate_query = f"TRUNCATE TABLE {table};"
    cursor.execute(truncate_query)
conn.commit()

In [ ]:
insert_query = """
    INSERT INTO categorias (id, name) VALUES (%s, %s)
"""

try:
    for index, row in categorias.iterrows():
        cursor.execute(insert_query, (
            row['id'], 
            row['name']
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()

In [ ]:
insert_query = """
    INSERT INTO items (id, order_id, product_id, quantity, total_price) VALUES (%s, %s, %s, %s, %s)
"""

try:    
    for index, row in items.iterrows():
        cursor.execute(insert_query, (
            int(row['id']), 
            int(row['order_id']), 
            int(row['product_id']), 
            int(row['quantity']), 
            float(row['total_price'])
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()

In [ ]:
insert_query = """
    INSERT INTO ordens (id, created_at, customer_id, status) VALUES (%s, %s, %s, %s)
"""

try:
    for index, row in ordens.iterrows():
        cursor.execute(insert_query, (
            row['id'], 
            row['created_at'], 
            row['customer_id'], 
            row['status']
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()

In [ ]:
insert_query = """
    INSERT INTO clientes (id, created_at, first_name, last_name, email, cell_phone, country, state, street, number, additionals) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    for index, row in clientes.iterrows():
        cursor.execute(insert_query, (
            row['id'], 
            row['created_at'],
            row['first_name'],
            row['last_name'],
            str(row['email']),
            row['cell_phone'],
            str(row['country']),
            str(row['state']),
            str(row['street']),
            str(row['number']),
            str(row['additionals']),
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()

In [ ]:
insert_query = """
    INSERT INTO produtos (id, name, price, id_category) VALUES (%s, %s, %s, %s)
"""

try:
    for index, row in produtos.iterrows():
        cursor.execute(insert_query, (
            row['id'], 
            row['name'],
            row['price'],
            row['id_category'],
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()

In [ ]:
# Fechar conexão e cursor
cursor.close()
conn.close()